In [1]:
import pandas as pd
import os
import numpy as np

def make_df(data_path):
    df = pd.read_csv(data_path, compression='gzip')
    id="Gene-ID" if "RCC7" in data_path else "Gene_ID"
    df = df.drop(columns=[id]).set_index('Symbol')

    target_genes = pd.read_csv("/home/sr933/rcc/data/Target_genes.csv")

    
    # Assuming the Target_genes.csv contains a column named 'gene' with row indices
    target_indices = target_genes['Genes'].values

    # Filter X_data to only include rows present in target_indices
    filtered_df = df.loc[target_indices]
    
    
    return filtered_df
main_folder="/data/sr933/scRCC validation/GSE156632_RAW"

# Initialize lists to store data and labels
data = []
labels = []

# Loop through files in the main folder
for file in os.listdir(main_folder):
    print(file)
    data_path = os.path.join(main_folder, file)
    
    # Create a numpy array from the file
    ar = make_df(data_path)  # Ensure `make_nparray` is properly implemented
    data.append(ar.to_numpy().T)
    
    # Step 2: Extract column names from the first CSV
    column_names = ar.columns
    
    # Step 2: Extract the base name of the first file (without extension)
    file_base_name = os.path.splitext(os.path.basename(file))[0]

    # Step 3: Construct the path to the second CSV file
    folder_path = "/data/sr933/scRCC validation/processed data"
    second_csv_path = os.path.join(folder_path, f"{file_base_name}.gzcell_annotation.csv")

    # Step 3: Read the second CSV file that contains the column names and cell-type labels
    cell_type_data = pd.read_csv(second_csv_path)

    # Step 4: Create a dictionary to map column names to labels
    # Assuming the first column in the second CSV is the column name and the last column is the label
    label_dict = pd.Series(cell_type_data.iloc[:, -1].values, index=cell_type_data.iloc[:, 0].values).to_dict()

    # Step 5: Create a list of labels corresponding to the columns of ar
    labels.append( [label_dict[column_name] for column_name in column_names])


# Combine all data and labels into arrays
data = np.concatenate(data, axis=0)
labels = np.concatenate(labels)
# Validate shapes
print("Data shape:", data.shape)
print("Labels shape:", labels.shape)

GSM4735364_RCC1t.csv.gz
GSM4735365_RCC1n.csv.gz
GSM4735366_RCC2t.csv.gz


KeyboardInterrupt: 

In [3]:
import pickle
validation_data={"X_test": data, "y_test":labels}
# Path to save the pickle file
pickle_file_path = "/data/sr933/scRCC validation/processed data/scRCC_validation_dataset_china.pkl"  # Update with your desired path

# Save the validation_data dictionary as a pickle file
with open(pickle_file_path, "wb") as pkl_file:
    pickle.dump(validation_data, pkl_file)